<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8ad08e506843217b2ae9300911b85d2e4f80b8099bea3244426c648903e50943
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-07 12:24:03
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: -17.86 K (-0.13%)
Current PnL: -20.65 L (-13.71%)
CY Booked + Current PnL: -9.21 L (-6.11%)
-------------------
Total profit:  1.42 L
Total loss:  -22.07 L
-------------------
Total Booked + Current PnL: 17.77 L (14.36%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.12%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.16 L (62.55%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.91%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.83,62.0,H-MC,13.71,114814.0,14310.0,14800.0,-0.24,14.24,12.89,28.96,79.0,0.97,0.82,53.04,MH,ATH,METALS
14,BLUESTARCO,2080.00,10.55,51.0,H-MC,2.97,190800.0,26130.0,17191.0,0.80,15.87,9.01,26.31,89.0,1.52,1.36,25.12,X40N,NTT,AC
77,TTKPRESTIG,770.00,93.60,40.0,M-SC,1.35,83408.0,-17369.0,17466.0,0.20,-17.24,20.94,0.09,245.0,-0.99,0.59,9.58,OX40N,NTT,DURABLES
19,CIPLA,1662.74,-22.52,42.0,H-LC,3.90,206788.0,2288.0,21010.0,-0.24,1.12,10.16,11.39,10.0,0.11,1.47,10.17,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-13.66,52.0,H-LC,3.34,216388.0,6022.0,22764.0,0.88,2.86,10.52,13.68,37.0,0.26,1.54,19.86,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,STARHEALTH,761.0,15.14,66.0,H-SC,5.82,256523.0,-24600.0,157454.0,3.07,-8.75,61.38,47.26,144.0,-0.16,1.83,37.44,XY24,NTT,INSURANCE
58,REPCOHOME,880.0,-59.76,54.0,H-SC,3.48,229896.0,-56609.0,313946.0,2.54,-19.76,136.56,89.82,134.0,-0.18,1.64,21.63,XY24,NTT,FINANCE
6,ASIANTILES,137.0,6566.67,47.0,L-SC,6.54,73135.0,-20675.0,97291.0,2.14,-22.04,133.03,81.67,269.0,-0.21,0.52,41.63,XR,NTT,CERAMICS
85,WHIRLPOOL,2270.0,-46.47,35.0,M-SC,1.09,90578.0,-920.0,79672.0,2.06,-1.01,87.96,86.07,223.0,-0.01,0.65,31.53,XR,NTT,DURABLES
44,JIOFIN,387.0,-1.23,54.0,H-LC,12.75,268277.0,912.0,64547.0,1.86,0.34,24.06,24.48,48.0,0.01,1.91,55.50,XY24,BTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,-8.25,29.0,H-SC,6.75,109217.0,-16080.0,75010.0,-2.17,-12.83,68.68,47.03,141.0,-0.21,0.78,36.75,MH,ATH,POWER
69,SURYODAY,240.00,62.28,61.0,H-SC,6.63,147992.0,-31079.0,96802.0,-2.10,-17.36,65.41,36.71,135.0,-0.32,1.05,46.84,XR,NTT,BANKS
38,INDIGOPNTS,1408.00,100.33,35.0,M-SC,2.12,131316.0,-43243.0,43282.0,-2.05,-24.77,32.96,0.02,221.0,-1.00,0.94,14.22,OX40N,NTT,PAINTS
73,TATAMOTORS,1065.00,-19.53,51.0,M-LC,2.28,254173.0,-22350.0,132424.0,-1.75,-8.08,52.10,39.81,54.0,-0.17,1.81,21.81,XY24,NTT,AUTO
7,ATULAUTO,844.00,3543.24,48.0,M-SC,6.62,163608.0,-33524.0,123344.0,-1.55,-17.01,75.39,45.57,236.0,-0.27,1.17,15.99,XY24,NTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1233.33,65.0,M-SC,20.89,181289.0,146.0,45322.0,-1.28,0.08,25.00,25.10,235.0,0.00,1.29,45.28,XY24,NTT,HEALTHCARE
60,SAIL,228.0,43.54,49.0,M-MC,12.32,226202.0,1240.0,166417.0,-0.95,0.55,73.57,74.53,192.0,0.01,1.61,32.98,XY24,BTT,STEEL
35,ICICIPRULI,790.0,-21.00,49.0,H-MC,2.06,135962.0,174.0,42583.0,0.19,0.13,31.32,31.48,107.0,0.00,0.97,12.26,X40,ATH,INSURANCE
86,WIPRO,420.0,-14.25,44.0,M-LC,5.71,150362.0,-583.0,110035.0,0.16,-0.39,73.18,72.51,53.0,-0.01,1.07,5.61,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,107.70,65.0,H-SC,13.51,131166.0,-11313.0,30837.0,-0.63,-7.94,23.51,13.70,163.0,-0.37,0.93,54.81,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.42,55.0,H-SC,2.11,224550.0,-45117.0,81444.0,-0.59,-16.73,36.27,13.47,138.0,-0.55,1.60,14.18,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.60,40.0,M-SC,1.35,83408.0,-17369.0,17466.0,0.20,-17.24,20.94,0.09,245.0,-0.99,0.59,9.58,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.67,39.0,H-SC,1.52,141024.0,-34879.0,76900.0,0.14,-19.83,54.53,23.89,149.0,-0.45,1.00,21.79,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-25.22,39.0,H-MC,6.28,101052.0,-30783.0,73010.0,-1.40,-23.35,72.25,32.03,98.0,-0.42,0.72,13.19,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1233.33,65.0,M-SC,20.89,181289.0,146.0,45322.0,-1.28,0.08,25.00,25.10,235.0,0.00,1.29,45.28,XY24,NTT,HEALTHCARE
60,SAIL,228.00,43.54,49.0,M-MC,12.32,226202.0,1240.0,166417.0,-0.95,0.55,73.57,74.53,192.0,0.01,1.61,32.98,XY24,BTT,STEEL
32,HINDZINC,730.22,29.80,64.0,M-LC,6.18,211323.0,6247.0,106317.0,-1.16,3.05,50.31,54.89,52.0,0.06,1.51,27.88,X5K,ATH,METALS
14,BLUESTARCO,2080.00,10.55,51.0,H-MC,2.97,190800.0,26130.0,17191.0,0.80,15.87,9.01,26.31,89.0,1.52,1.36,25.12,X40N,NTT,AC
84,VOLTAS,1530.00,-2.06,49.0,H-MC,1.43,205500.0,13758.0,24002.0,0.24,7.18,11.68,19.69,99.0,0.57,1.46,14.46,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.83,62.0,H-MC,13.71,114814.0,14310.0,14800.0,-0.24,14.24,12.89,28.96,79.0,0.97,0.82,53.04,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,71.0,H-MC,10.86,195495.0,15687.0,96731.0,0.85,8.72,49.48,62.52,88.0,0.16,1.39,45.79,XR,NTT,BANKS
32,HINDZINC,730.22,29.80,64.0,M-LC,6.18,211323.0,6247.0,106317.0,-1.16,3.05,50.31,54.89,52.0,0.06,1.51,27.88,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.83,36.0,H-SC,9.35,126140.0,2804.0,128827.0,0.17,2.27,102.13,106.72,119.0,0.02,0.90,26.18,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-46.47,35.0,M-SC,1.09,90578.0,-920.0,79672.0,2.06,-1.01,87.96,86.07,223.0,-0.01,0.65,31.53,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.30,30.0,H-LC,9.96,279763.0,-36079.0,148750.0,-0.10,-11.42,53.17,35.67,5.0,-0.24,1.99,0.85,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-17.90,33.0,H-LC,5.77,207134.0,-44634.0,97850.0,-0.04,-17.73,47.24,21.13,27.0,-0.46,1.48,11.42,X40,ATH,PAINTS
42,ITC,452.00,-39.61,41.0,H-LC,2.16,195698.0,-4440.0,25323.0,-0.14,-2.22,12.94,10.44,4.0,-0.18,1.39,3.22,X40,NTT,FMCG
31,HINDUNILVR,2922.00,-10.50,41.0,H-LC,8.00,251450.0,-5613.0,40760.0,-1.07,-2.18,16.21,13.67,24.0,-0.14,1.79,17.78,XY25,NTT,FMCG
74,TCS,4406.34,-30.29,42.0,H-LC,12.20,277873.0,-68083.0,136324.0,-1.08,-19.68,49.06,19.73,1.0,-0.50,1.98,2.34,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,0.60,55.0,H-LC,1.99,161820.0,-24275.0,71751.0,-0.47,-13.04,44.34,25.51,15.0,-0.34,1.15,19.25,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.61,41.0,H-LC,2.16,195698.0,-4440.0,25323.0,-0.14,-2.22,12.94,10.44,4.0,-0.18,1.39,3.22,X40,NTT,FMCG
48,LTIM,7230.2,-4.34,54.0,H-LC,2.36,237218.0,-8933.0,88150.0,-0.05,-3.63,37.16,32.18,16.0,-0.10,1.69,31.11,X200,ATH,IT
51,NESTLEIND,1377.0,-10.01,53.0,H-LC,3.06,276892.0,11466.0,45327.0,0.14,4.32,16.37,21.40,11.0,0.25,1.97,11.25,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.66,52.0,H-LC,3.34,216388.0,6022.0,22764.0,0.88,2.86,10.52,13.68,37.0,0.26,1.54,19.86,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.30,30.0,H-LC,9.96,279763.0,-36079.0,148750.0,-0.10,-11.42,53.17,35.67,5.0,-0.24,1.99,0.85,X40N,ATH,FMCG
74,TCS,4406.34,-30.29,42.0,H-LC,12.20,277873.0,-68083.0,136324.0,-1.08,-19.68,49.06,19.73,1.0,-0.50,1.98,2.34,X40,ATH,IT
42,ITC,452.00,-39.61,41.0,H-LC,2.16,195698.0,-4440.0,25323.0,-0.14,-2.22,12.94,10.44,4.0,-0.18,1.39,3.22,X40,NTT,FMCG
40,INFY,2275.00,-20.32,44.0,H-LC,8.09,310469.0,-2419.0,174111.0,-1.25,-0.77,56.08,54.87,3.0,-0.01,2.21,5.82,X40,BTT,IT
80,UNITDSPR,1644.00,-11.52,56.0,H-LC,6.88,232097.0,-2047.0,50667.0,-0.85,-0.87,21.83,20.77,86.0,-0.04,1.65,5.98,X40N,NTT,BREWERIES


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.60,55.0,H-LC,1.99,161820.0,-24275.0,71751.0,-0.47,-13.04,44.34,25.51,15.0,-0.34,1.15,19.25,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,41.0,H-LC,2.16,195698.0,-4440.0,25323.0,-0.14,-2.22,12.94,10.44,4.0,-0.18,1.39,3.22,X40,NTT,FMCG
19,CIPLA,1662.74,-22.52,42.0,H-LC,3.90,206788.0,2288.0,21010.0,-0.24,1.12,10.16,11.39,10.0,0.11,1.47,10.17,X40N,ATH,PHARMA
5,ASIANPAINT,3465.66,-17.90,33.0,H-LC,5.77,207134.0,-44634.0,97850.0,-0.04,-17.73,47.24,21.13,27.0,-0.46,1.48,11.42,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.66,52.0,H-LC,3.34,216388.0,6022.0,22764.0,0.88,2.86,10.52,13.68,37.0,0.26,1.54,19.86,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-30.29,42.0,H-LC,12.20,277873.0,-68083.0,136324.0,-1.08,-19.68,49.06,19.73,1.0,-0.50,1.98,2.34,X40,ATH,IT
40,INFY,2275.00,-20.32,44.0,H-LC,8.09,310469.0,-2419.0,174111.0,-1.25,-0.77,56.08,54.87,3.0,-0.01,2.21,5.82,X40,BTT,IT
42,ITC,452.00,-39.61,41.0,H-LC,2.16,195698.0,-4440.0,25323.0,-0.14,-2.22,12.94,10.44,4.0,-0.18,1.39,3.22,X40,NTT,FMCG
83,VBL,671.64,-22.30,30.0,H-LC,9.96,279763.0,-36079.0,148750.0,-0.10,-11.42,53.17,35.67,5.0,-0.24,1.99,0.85,X40N,ATH,FMCG
1,ABB,7934.00,-39.77,51.0,H-LC,7.51,251808.0,-9811.0,129026.0,0.54,-3.75,51.24,45.57,7.0,-0.08,1.79,7.15,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,31.62,79104.0,-34445.0,74492.0,-0.07,-30.33,94.17,35.27,268.0,-0.46,0.56,93.61,XR,NTT,HOTELS
49,MASFIN,398.61,-19.60,45.0,H-SC,11.44,91620.0,-6360.0,27962.0,-0.89,-6.49,30.52,22.05,152.0,-0.23,0.65,33.18,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.83,62.0,H-MC,13.71,114814.0,14310.0,14800.0,-0.24,14.24,12.89,28.96,79.0,0.97,0.82,53.04,MH,ATH,METALS
78,UJJIVANSFB,60.00,107.70,65.0,H-SC,13.51,131166.0,-11313.0,30837.0,-0.63,-7.94,23.51,13.70,163.0,-0.37,0.93,54.81,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-185.23,68.0,M-SC,12.57,146394.0,-2826.0,147009.0,1.70,-1.89,100.42,96.62,208.0,-0.02,1.04,58.43,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.0,-185.23,68.0,M-SC,12.57,146394.0,-2826.0,147009.0,1.70,-1.89,100.42,96.62,208.0,-0.02,1.04,58.43,XY25,NTT,FINANCE
69,SURYODAY,240.0,62.28,61.0,H-SC,6.63,147992.0,-31079.0,96802.0,-2.10,-17.36,65.41,36.71,135.0,-0.32,1.05,46.84,XR,NTT,BANKS
26,GREENPANEL,537.0,233.98,60.0,M-SC,3.70,149035.0,-34043.0,113013.0,1.06,-18.59,75.83,43.14,230.0,-0.30,1.06,40.19,XY24,NTT,MISC
11,BANKINDIA,190.0,-24.22,71.0,H-MC,10.86,195495.0,15687.0,96731.0,0.85,8.72,49.48,62.52,88.0,0.16,1.39,45.79,XR,NTT,BANKS
64,SHALBY,327.0,1233.33,65.0,M-SC,20.89,181289.0,146.0,45322.0,-1.28,0.08,25.00,25.10,235.0,0.00,1.29,45.28,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.09
1,25,43.28
2,50,74.97


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.07
LC    32.13
MC    23.74
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.99
X40      14.56
X40N     11.18
XR       10.97
XY25     10.40
AR        8.20
OX40N     7.70
X200      1.69
MH        1.60
X5K       1.51
SR        1.14
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.95
H-LC    25.61
H-MC    20.52
M-SC    13.70
M-LC     5.51
M-MC     2.92
L-SC     1.42
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.76,-6.18,41.17
IT,12.59,-18.90,83.29
FINANCE,10.50,-14.48,69.77
MISC,6.95,-17.79,83.06
BANKS,6.36,-11.48,70.13
PAINTS,5.55,-18.65,36.33
ELECTRICAL,5.37,-9.06,47.32
INSURANCE,3.82,-3.29,41.89
AC,3.51,1.82,14.62


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3223272.0,22
XR,1366973.0,14
AR,1177919.0,9
X40,795840.0,10
X40N,664699.0,9
OX40N,564998.0,10
XY25,475872.0,7
SR,262127.0,2
X5K,106317.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3725213.0,29
M-SC,1508355.0,17
H-MC,1273712.0,15
H-LC,1195949.0,15
M-LC,377246.0,4
M-MC,372813.0,2
L-SC,261048.0,3
L-MC,59846.0,1
L-LC,41795.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1208018.0      6
M-SC       XY24       849029.0      7
H-SC       AR         808796.0      5
           XR         800397.0      7
H-MC       XY24       567971.0      4
H-LC       X40        513197.0      5
M-MC       XY24       372813.0      2
H-SC       X40N       311543.0      4
           SR         262127.0      2
           OX40N      259322.0      4
H-LC       X40N       220427.0      3
H-MC       X40        210254.0      4
H-LC       AR         200777.0      2
H-MC       XY25       178217.0      2
L-SC       XR         171783.0      2
M-SC       AR         168346.0      2
M-LC       XY24       160894.0      2
M-SC       XY25       147009.0      1
           OX40N      143401.0      4
H-MC       X40N       132729.0      2
M-SC       XR         128181.0      2
M-LC       XR         110035.0      1
H-LC       XY25       108851.0      3
M-LC       X5K        106317.0      1
H-MC       XR          96731.0      1
L-SC       OX40N       89265.0      1
H-LC       X200        88150.0      1
H-SC       MH          75010.0      1
H-MC       OX40N       73010.0      1
M-SC       X40         72389.0      1
H-LC       XY24        64547.0      1
L-MC       XR          59846.0      1
L-LC       XY25        41795.0      1
H-MC       MH          14800.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
